In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(x)
        x = torch.relu(self.fc2(x))
        x = torch.relu(x)
        x = self.fc3(x)
        return F.sigmoid(x)

In [75]:
def train_eval(XX_path, yy_path, hidden_layer1, batch_size, learning_rate):
  current_feature = 'text'
  if 'visual' in XX_path:
    current_feature = 'visual'
  elif'audio' in XX_path:
      current_feature = 'audio'

  XX = pd.read_csv(XX_path)
  yy = pd.read_csv(yy_path)
  X = XX.values
  y = yy.values.reshape(-1)
  # yy=pd.read_csv('/content/drive/MyDrive/CS535/HW2/text_y.csv')
  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  input_size = X.shape[1]
  output_size = 4

  class_labels = np.array([0,1,2,3])
  class_weights = compute_class_weight('balanced', classes=class_labels, y=y)
  class_weights = torch.tensor(class_weights, dtype=torch.float32)

  X = torch.tensor(X, dtype=torch.float32)
  y = torch.tensor(y, dtype=torch.float32)

  k_folds = 5
  kf = KFold(n_splits=k_folds, shuffle=True)
  num_epochs = 100

  best_F1 = 0
  best_hyperparameters = {}

  predicted_labels_all_fold = []
  true_labels_all_fold = []

  for fold, (train_index, test_index) in enumerate(kf.split(X)):
    predicted_labels_all = []
    true_labels_all = []

    # Split data into train and test sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Create DataLoader for training and testing sets
    train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
    test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Create an instance of the model
    model = MLP(input_size, hidden_layer1, 64, output_size)

    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    for epoch in range(num_epochs):
      for feature, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(feature)
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

      # if (epoch+1) % 10 == 0:
      #   print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

    # Evaluate the model on the test set
    model.eval()

    with torch.no_grad():
      for feature, labels in test_loader:
        outputs = model(feature)
        _, predicted = torch.max(outputs, 1)
        predicted_labels_all.append(predicted)
        true_labels_all.append(labels)
        predicted_labels_all_fold.append(predicted)
        true_labels_all_fold.append(labels)

    true_labels_all = torch.cat(true_labels_all, dim=0)
    predicted_labels_all = torch.cat(predicted_labels_all, dim=0)
    f1 = f1_score(true_labels_all, predicted_labels_all, average='micro')

    cf_matrix = confusion_matrix(true_labels_all, predicted_labels_all, labels = class_labels)
    print(f'Fold: {fold}, F1: {f1}')
    print(cf_matrix)

    if f1 > best_F1:
      torch.save(model, '/content/drive/MyDrive/CS535/HW2/' + current_feature + '.pth')
      best_F1 = f1

  # Calculate F1 across all folds
  true_labels_all_fold = torch.cat(true_labels_all_fold, dim=0)
  predicted_labels_all_fold = torch.cat(predicted_labels_all_fold, dim=0)
  f1_all_fold = f1_score(true_labels_all_fold, predicted_labels_all_fold, average='micro')
  print(f'F1 for 5 folds: {f1_all_fold}')

In [ ]:
def find_hyperparameter(XX_path, yy_path):

  current_feature = 'text'
  if 'visual' in XX_path:
    current_feature = 'visual'
  elif'audio' in XX_path:
      current_feature = 'audio'

  XX = pd.read_csv(XX_path)
  yy = pd.read_csv(yy_path)

  X = XX.values
  y = yy.values.reshape(-1)

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  input_size = X.shape[1]
  output_size = 4

  class_labels = np.array([0,1,2,3])
  class_weights = compute_class_weight('balanced', classes=class_labels, y=y)
  class_weights = torch.tensor(class_weights, dtype=torch.float32)

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # PyTorch Tensor
  X_train = torch.tensor(X_train, dtype=torch.float32)
  y_train = torch.tensor(y_train, dtype=torch.long)
  X_test = torch.tensor(X_test, dtype=torch.float32)
  y_test = torch.tensor(y_test, dtype=torch.long)

  train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
  test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

  num_epochs = 100

  # Perform Grid Search
  import itertools
  hidden_size1 = [512, 256, 128]
  hidden_size2 = [64]
  learning_rates = [0.0005, 0.001, 0.005]
  batch_sizes = [128, 64, 32]
  # hidden_size1 = [512]
  # hidden_size2 = [128]
  # learning_rates = [0.0005]
  # batch_sizes = [64]
  best_F1 = 0
  best_hyperparameters = {}

  for size1, size2, learning_rate, batch_size in itertools.product(hidden_size1, hidden_size2, learning_rates, batch_sizes):
    print(f'Size1: {size1}, learning_rate: {learning_rate}, batch_size: {batch_size}')
    predicted_labels_all = []
    true_labels_all = []

    # Create an instance of the model
    model = MLP(input_size, size1, size2, output_size)

    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Train the model
    for epoch in range(num_epochs):
      for feature, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(feature)
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

    # Evaluate the model on the test set
    model.eval()

    with torch.no_grad():
      for feature, labels in test_loader:
        outputs = model(feature)
        _, predicted = torch.max(outputs, 1)
        predicted_labels_all.append(predicted)
        true_labels_all.append(labels)

    true_labels_all = torch.cat(true_labels_all, dim=0)
    predicted_labels_all = torch.cat(predicted_labels_all, dim=0)
    F1 = f1_score(true_labels_all, predicted_labels_all, average='micro')
    print(f'F1: {F1}')

    if F1 > best_F1:
      best_hyperparameters['hidden_size1'] = size1
      best_hyperparameters['learning_rate'] = learning_rate
      best_hyperparameters['batch_size'] = batch_size
      best_F1 = F1

  print(current_feature)
  print(best_hyperparameters)
  print(f'Best F1: {best_F1}')

In [ ]:
Visual_X_path='/content/drive/MyDrive/CS535/HW2/visual_x.csv'
Visual_y_path='/content/drive/MyDrive/CS535/HW2/visual_y.csv'
find_hyperparameter(Visual_X_path, Visual_y_path)

Size1: 512, learning_rate: 0.0005, batch_size: 128
F1: 0.5298507462686567
Size1: 512, learning_rate: 0.0005, batch_size: 64
F1: 0.5522388059701493
Size1: 512, learning_rate: 0.0005, batch_size: 32
F1: 0.5522388059701493
Size1: 512, learning_rate: 0.001, batch_size: 128
F1: 0.5261194029850746
Size1: 512, learning_rate: 0.001, batch_size: 64
F1: 0.5597014925373134
Size1: 512, learning_rate: 0.001, batch_size: 32
F1: 0.5186567164179104
Size1: 512, learning_rate: 0.005, batch_size: 128
F1: 0.35074626865671643
Size1: 512, learning_rate: 0.005, batch_size: 64
F1: 0.39552238805970147
Size1: 512, learning_rate: 0.005, batch_size: 32
F1: 0.22761194029850745
Size1: 256, learning_rate: 0.0005, batch_size: 128
F1: 0.5485074626865671
Size1: 256, learning_rate: 0.0005, batch_size: 64
F1: 0.582089552238806
Size1: 256, learning_rate: 0.0005, batch_size: 32
F1: 0.5634328358208955
Size1: 256, learning_rate: 0.001, batch_size: 128
F1: 0.5410447761194029
Size1: 256, learning_rate: 0.001, batch_size: 64
F1

visual
{'hidden_size1': 256, 'learning_rate': 0.0005, 'batch_size': 64}
Best F1: 0.582089552238806

In [76]:
train_eval(Visual_X_path, Visual_y_path, 256, 64, 0.0005)

Fold: 0, F1: 0.5559701492537313
[[35  7  3 24]
 [ 4 30  3 11]
 [10 11 11 11]
 [14 12  9 73]]
Fold: 1, F1: 0.5543071161048689
[[34  9  4 15]
 [10 33  5 18]
 [ 6  6  9 13]
 [14 13  6 72]]
Fold: 2, F1: 0.5917602996254682
[[37  6  2 22]
 [12 32  5 13]
 [ 7  3 15  8]
 [12 12  7 74]]
Fold: 3, F1: 0.5468164794007491
[[42  7  7  7]
 [16 29  4 24]
 [ 7  7 16  7]
 [22 10  3 59]]
Fold: 4, F1: 0.4756554307116105
[[28  9 10 20]
 [11 25  7 16]
 [ 5  4 10 14]
 [20 17  7 64]]
F1 for 5 folds: 0.5449101796407185


In [ ]:
Audio_X_path='/content/drive/MyDrive/CS535/HW2/audio_x.csv'
Audio_y_path='/content/drive/MyDrive/CS535/HW2/audio_y.csv'
find_hyperparameter(Audio_X_path, Audio_y_path)

Size1: 512, learning_rate: 0.0005, batch_size: 128
F1: 0.5522388059701493
Size1: 512, learning_rate: 0.0005, batch_size: 64
F1: 0.5447761194029851
Size1: 512, learning_rate: 0.0005, batch_size: 32
F1: 0.5559701492537313
Size1: 512, learning_rate: 0.001, batch_size: 128
F1: 0.5410447761194029
Size1: 512, learning_rate: 0.001, batch_size: 64
F1: 0.5671641791044776
Size1: 512, learning_rate: 0.001, batch_size: 32
F1: 0.5149253731343284
Size1: 512, learning_rate: 0.005, batch_size: 128
F1: 0.5335820895522388
Size1: 512, learning_rate: 0.005, batch_size: 64
F1: 0.5746268656716418
Size1: 512, learning_rate: 0.005, batch_size: 32
F1: 0.5298507462686567
Size1: 256, learning_rate: 0.0005, batch_size: 128
F1: 0.5522388059701493
Size1: 256, learning_rate: 0.0005, batch_size: 64
F1: 0.5783582089552238
Size1: 256, learning_rate: 0.0005, batch_size: 32
F1: 0.5410447761194029
Size1: 256, learning_rate: 0.001, batch_size: 128
F1: 0.5559701492537313
Size1: 256, learning_rate: 0.001, batch_size: 64
F1: 

audio
{'hidden_size1': 256, 'learning_rate': 0.0005, 'batch_size': 64}
Best F1: 0.5783582089552238

In [78]:
train_eval(Audio_X_path, Audio_y_path, 256, 64, 0.0005)

Fold: 0, F1: 0.5149253731343284
[[43  4  5 17]
 [ 2 36  5 21]
 [ 5  7  4 16]
 [ 7 25 16 55]]
Fold: 1, F1: 0.5655430711610487
[[47  6  4 13]
 [ 3 40  7 14]
 [ 6  4  6 21]
 [12 15 11 58]]
Fold: 2, F1: 0.5243445692883895
[[34  2  9 16]
 [ 4 43  8 10]
 [ 6  8  2 17]
 [16 22  9 61]]
Fold: 3, F1: 0.5243445692883895
[[47  2  5 15]
 [ 8 30  6 16]
 [ 6  6  8 17]
 [14 16 16 55]]
Fold: 4, F1: 0.5468164794007491
[[43  3  3 10]
 [ 2 34  6 13]
 [ 2 13  5 21]
 [15 13 20 64]]
F1 for 5 folds: 0.5351796407185628


In [ ]:
Text_X_path='/content/drive/MyDrive/CS535/HW2/text_x.csv'
Text_y_path='/content/drive/MyDrive/CS535/HW2/text_y.csv'
find_hyperparameter(Text_X_path, Text_y_path)

Size1: 512, learning_rate: 0.0005, batch_size: 128
F1: 0.6044776119402985
Size1: 512, learning_rate: 0.0005, batch_size: 64
F1: 0.6305970149253731
Size1: 512, learning_rate: 0.0005, batch_size: 32
F1: 0.6268656716417911
Size1: 512, learning_rate: 0.001, batch_size: 128
F1: 0.6268656716417911
Size1: 512, learning_rate: 0.001, batch_size: 64
F1: 0.6156716417910447
Size1: 512, learning_rate: 0.001, batch_size: 32
F1: 0.6119402985074627
Size1: 512, learning_rate: 0.005, batch_size: 128
F1: 0.6231343283582089
Size1: 512, learning_rate: 0.005, batch_size: 64
F1: 0.5373134328358209
Size1: 512, learning_rate: 0.005, batch_size: 32
F1: 0.40298507462686567
Size1: 256, learning_rate: 0.0005, batch_size: 128
F1: 0.6380597014925373
Size1: 256, learning_rate: 0.0005, batch_size: 64
F1: 0.6492537313432836
Size1: 256, learning_rate: 0.0005, batch_size: 32
F1: 0.5970149253731343
Size1: 256, learning_rate: 0.001, batch_size: 128
F1: 0.6268656716417911
Size1: 256, learning_rate: 0.001, batch_size: 64
F1:

text
{'hidden_size1': 256, 'learning_rate': 0.0005, 'batch_size': 64}
Best F1: 0.6492537313432836

In [79]:
train_eval(Text_X_path, Text_y_path, 256, 64, 0.0005)

Fold: 0, F1: 0.6156716417910447
[[47  3  0 11]
 [ 6 42  6 11]
 [ 5  5 11 14]
 [15 16 11 65]]
Fold: 1, F1: 0.6367041198501873
[[42  4  3 14]
 [ 7 45  7 14]
 [ 6  6 13  6]
 [10 15  5 70]]
Fold: 2, F1: 0.5805243445692884
[[48 10  3 18]
 [ 6 26  4  9]
 [ 3  3 16 18]
 [13 19  6 65]]
Fold: 3, F1: 0.6067415730337079
[[44  3  3 15]
 [ 7 35  4 13]
 [ 3  5 13 16]
 [12 16  8 70]]
Fold: 4, F1: 0.6367041198501873
[[46  3  5  6]
 [ 7 46  3 10]
 [ 4  7 13 13]
 [15  8 16 65]]
F1 for 5 folds: 0.6152694610778443
